In [5]:
# autoreload to cast any changes to the module files into the notebook
%load_ext autoreload
%autoreload 2
import re, math, os, csv, timeit
import numpy as np
from datetime import date 

In [65]:
# configurations
# this script is applicatble only for ring-molecules
NUM_CARBON = 8
NUM_ATOMS = NUM_CARBON + 1*4 + 2*4  # C*8 + H*4 + (2H)*4

# represent C-H bonds
# C6-H9
# C7-H10
# C8-H11
# C1-H12
# C5-H13 & C5-H14
# C4-H15 & C4-H16
# C3-H17 & C3-H20
# C2-H18 & C2-H19
carbon_list = [6,7,8,1,5,5,4,4,3,3,2,2]
hydrogen_list = [9,10,11,12,13,14,15,16,17,20,18,19]

In [66]:
# Program flow
# --------------------------- Preprocess Data -------------------------
# loop file 
    # catch patterns
    # read 20 cooridnates into a block (list of coordinates)
    # append to list_of_blocks
    
# --------------------------- Perform Computation -------------------------
# loop through list of blocks 
# for index in len(list_of_blocks)
    # compute(block)
        
    # create new entries in list_of_dictionaries "Traj : 1, C1-C2: (double), C2-C3 (double), ... "
        
# --------------------------- Write to csv ---------------------------

In [67]:
# --------------------------- Preprocess Data -------------------------

def get_lines_with_coordinates(fileName):
    ''' take in xyz file & grab lines with coordinates
        Input: fileName
        Output: a list of lines with coordinates
        '''
    searchfile = open(fileName, "r",encoding='utf-8')
    line_list = searchfile.readlines()
    line_list = [line for line in line_list if line != ' {} \n'.format(NUM_ATOMS)]    # strip all lines with " NUM_ATOMS \n"
    return line_list


def split_into_trajectory_blocks(line_list):
    ''' convert file line_list into trajectory_blocks
        Input: line_list
        Output: a list, trajectory_blocks
        '''
    trajectory_blocks = []
    for index, line in enumerate(line_list):
        if re.search(r'TRAJ\d+', line):
            # create new block for each trajectory
            trajectory_coordinates = get_trajectory_data(index,line_list)

            # append to list
            trajectory_blocks.append(trajectory_coordinates)            
    return trajectory_blocks


def get_trajectory_data(index, line_list):
    ''' read a trajetory --> if(match regex | EOF), return 
        Input: index, list_of_lines_in_file
        Output: a list of coordinates for that block 
        '''
    trajectory_coordinates = [line_list[index][:-1]] # grab trajectory #
    for line in line_list[index+1:]:
        if (re.search(r'TRAJ\d+', line)):
            break
        else:
            atom,x,y,z = line.split()
            trajectory_coordinates.append([float(x), float(y), float(z)])            
    return trajectory_coordinates


def preprocess_data(fileName):    
    ''' Input: filename
        Output: a list of trajectory blocks_in_file, 
                each block representing 1 trajectory
        '''
    line_list = get_lines_with_coordinates(fileName)
    traj_blocks_in_file = split_into_trajectory_blocks(line_list)
    return traj_blocks_in_file   

![dihedral illustration](https://tinyurl.com/y8l79dlh)

In [68]:
def compute_C_C_C_angle(*coords):
    ''' compute C-C-C angles
        Input: 2 lines of coordinates
        Output: angle (degree / radian) 
        '''
    return 1
        
# compute dihedral angles
# https://azevedolab.net/resources/dihedral_angle.pdf


# This one starts with two cross products to get a vector perpendicular to
# b2 and b1 and another perpendicular to b2 and b3. The angle between those vectors
# is the dihedral angle.

def compute_dihedral_angle(*coords):
    ''' Input: a list of coordinates
        Output: dihedral_angle
    '''
    # build vectors, compute normal-vectors of intersecting planes, take sine from dot product
        
     
    b[0] *= -1
    v = np.array( [np.cross(v,b[1]) for v in [b[0], b[2]] ] )
    # Normalize vectors
    v /= np.sqrt(np.einsum('...i,...i', v, v)).reshape(-1,1)
    return np.degrees(np.arccos( v[0].dot(v[1]) ))

In [12]:
x = [[2,2,2], [1,2,3], [1,2,3]]
a = np.asarray(x, dtype = float) 
print(a)
x1 = a[0]
x2 = a[1]

np.subtract(x1, x2)

[[2. 2. 2.]
 [1. 2. 3.]
 [1. 2. 3.]]


array([ 1.,  0., -1.])

In [69]:
# --------------------------- Perform Computation -------------------------
def compute_bondlength(*coords):
    ''' compute bondlength
        Input: 2 lines of coordinates
        Output: bondlength (float type)
        '''    
    coord1, coord2 = coords[0], coords[1]    
    return np.linalg.norm(coord2 - coord1) 


# **************REFRACTOR this****************
def compute(block):
    ''' compute block 
        Input: block
        Output: dictionary_for_trajectory 
                format :  "TRAJ : 1, C1-C2: (double), C2-C3 (double), ... "
        '''    
    dictionary_for_trajectory = {}
    dictionary_for_trajectory['TRAJ'] = re.findall('\d+', block[0])[0]   # findall the numbers in the string block[0], return a list --> index [0] to get string value
    # ---------------- C - C ----------------
    for carbon in list(range(1, NUM_CARBON+1)): 
        if carbon == NUM_CARBON:
            bond_title ="C{}-C{}".format(carbon,1)
            dictionary_for_trajectory[bond_title] = compute_bondlength(block[carbon],block[1])
        elif carbon != NUM_CARBON:     
            bond_title ="C{}-C{}".format(carbon,carbon+1)
            dictionary_for_trajectory[bond_title] = compute_bondlength(block[carbon],block[carbon + 1]) 
    # ---------------- C - H ----------------
    for carbon,hydro in zip(carbon_list,hydrogen_list):
        bond_title ="C{}-H{}".format(carbon,hydro)
        dictionary_for_trajectory[bond_title] = compute_bondlength(block[carbon],block[hydro])    
    return dictionary_for_trajectory

In [70]:
# REFRACTOR this, add perform_computation function & write_to_CSV function

def compute_geometric_params(inputFile):
    ''' extract from xyz, compute, write to csv
        Input: inputFile
        Output: csv
        '''
    # --------------------------- Preprocess Data -------------------------
    list_of_blocks_in_file = preprocess_data(inputFile)
    # --------------------------- Perform Computation -------------------------
    list_of_traj_dictionaries = []
    for block in list_of_blocks_in_file:

        # compute(block)
        dictionary_for_trajectory = compute(block)

        # create new entries in list_of_dictionaries "Traj : 1, C1-C2: (double), C2-C3 (double), ... "
        list_of_traj_dictionaries.append(dictionary_for_trajectory)
    # --------------------------- Write to csv ---------------------------
    date_computed = str(date.today())
    file_path = os.path.join(".", "output", date_computed + "_trajectory_bondlength" + ".csv")
    os.makedirs(os.path.dirname(file_path), exist_ok=True) # create 'output' folder if not existed yet

    with open(file_path, 'w', newline='', encoding='utf-8') as csvFile:
        field_names = list(list_of_traj_dictionaries[0].keys())
        writer = csv.DictWriter(csvFile, fieldnames=field_names, dialect='excel')

        writer.writeheader()
        for trajectory in list_of_traj_dictionaries:
            writer.writerow(trajectory)        

In [71]:
def main():
    compute_geometric_params(os.path.join('.','input','IC-angs-conflow.xyz'))

if __name__ == "__main__":
    main()  

In [72]:
# references
'''
http://google.github.io/styleguide/pyguide.html
https://visualgit.readthedocs.io/en/latest/pages/naming_convention.html
https://developers.google.com/edu/python/regular-expressions
'''

'\nhttp://google.github.io/styleguide/pyguide.html\nhttps://visualgit.readthedocs.io/en/latest/pages/naming_convention.html\nhttps://developers.google.com/edu/python/regular-expressions\n'